# Correr experimentos
En este archivo está el código para correr los experimentos y escribir los resultados en archivos CSV.
> Los archivos se guardan en la carpeta _resultados_.

In [1]:
import math, subprocess
import pandas as pd
import numpy as np
import shutil as sh
from IPython.display import display, clear_output
from os import listdir

La siguiente función sirve para correr el código sobre una instancia ejecutando un método en particular.


In [2]:
def correr_experimento(experimento, secuencial, cantThreadsLectura, cantThreadsMaximo, filePaths):

    # Crear proceso para ejecutar el codigo
    args = ["../build/Experimentos", experimento, secuencial, str(cantThreadsLectura), str(cantThreadsMaximo)]

    for path in filePaths:
        args.append(path)
        
    process = subprocess.Popen(args, stderr=subprocess.PIPE,stdout=subprocess.PIPE, stdin=subprocess.PIPE, universal_newlines = True)

    # Poner la instancia en la entrada estandar y leer salida de STDERR con el tiempo de ejecución.
    clog = process.communicate()[1] # communicate() devuelve una tupla (stdout, stderr)
    
    #clog = str.splitlines(clog)

    #print(clog)

    tiempo_de_ejecucion = clog

    # Correr experimento.
    exit_code = process.wait()

    # Verificar que el proceso no fallo.
    if exit_code != 0: raise(F"Hubo un error en la experimentacion.")
    
    return float(tiempo_de_ejecucion);

## Creamos una instancia propia
Esta instancia, pensada para los experimentos de máximo, posee una gran cantidad de elementos en cada bucket, de este modo buscamos que el costo de explorarlos para cada thread sea mayor al costo de crear el thread y así obtener mediciones en las que sea tangible la efectividad de la concurrencia por sobre secuencialidad.

In [9]:
abecedario = "abcdefghijklmnopqrstuvwxyz"
for i in range(200, 2001, 200):
    file = open(F"../data/miDiccionario{i}", "w")
    for letra in abecedario:
        s = ""
        for j in range(0, i):
            s+=letra
            file.write(s+"\n")
    file.write("z"+"\n")    
    file.close()

## Experimento máximo secuecial

In [11]:
columnas = ["n", "experimento", "es_secuencial", "cantThreadsLectura", "cantThreadsMaximo", "tiempo"];
filas = [];

n = 1;
experimento = "maximo";
es_secuencial = "Si";
cantThreadsLectura = 1;
cantThreadsMaximo = 1;
tiempos = []

for i in range(1, 11):
    for t in range(0, 5):
        filePaths = [F"../data/miDiccionario{i*200}"]
        analisis = correr_experimento(experimento, es_secuencial, cantThreadsLectura, cantThreadsMaximo, filePaths);
        tiempos.append(analisis);
    clear_output(wait=True)
    display('Experimento: ' + str(n))
    tiempo = np.median(tiempos);
    filas.append([n, experimento, es_secuencial, cantThreadsLectura, cantThreadsMaximo, tiempo]);
    n += 1;


df_resultado = pd.DataFrame(filas, columns=columnas);
df_resultado.to_csv("../resultados/resultado_maximo_secuencial.csv", index=False, header=True);




'Experimento: 10'

## Experimento máximo concurrente

In [3]:
columnas = ["n", "experimento", "es_secuencial", "cantThreadsLectura", "cantThreadsMaximo", "tiempo"];
filas = [];

experimento = "maximo";
es_secuencial = "No";
cantThreadsLectura = 1;
tiempos = []
for cantThreadsMaximo in range(1,27):
    for i in range(1, 11):
        n = i;
        for t in range(0, 5):
            filePaths = [F"../data/miDiccionario{i*200}"]
            analisis = correr_experimento(experimento, es_secuencial, cantThreadsLectura, cantThreadsMaximo, filePaths);
            tiempos.append(analisis);
        clear_output(wait=True)
        display('Experimento: ' + str(cantThreadsMaximo) + '.' + str(n))
        tiempo = np.median(tiempos);
        filas.append([n, experimento, es_secuencial, cantThreadsLectura, cantThreadsMaximo, tiempo]);


df_resultado = pd.DataFrame(filas, columns=columnas);
df_resultado.to_csv("../resultados/resultado_maximo_concurrente.csv", index=False, header=True);

'Experimento: 26.10'

## Experimento cargar archivos secuencial

In [18]:
columnas = ["n", "experimento", "es_secuencial", "cantThreadsLectura", "cantThreadsMaximo", "tiempo"];
filas = [];

n = 1;
experimento = "cargarArchivos";
es_secuencial = "Si";
cantThreadsLectura = 1;
cantThreadsMaximo = 1;
tiempos = []
filePaths = ["../data/corpus"]
for f in range(1,30):
    sh.copyfile("../data/corpus",F"../data/corpus{f}");
    filePaths.append(F"../data/corpus{f}")

for t in range(0, 5):
    analisis = correr_experimento(experimento, es_secuencial, cantThreadsLectura, cantThreadsMaximo, filePaths);
    tiempos.append(analisis);
clear_output(wait=True)
display('Experimento: ' + str(n))
tiempo = np.median(tiempos);
filas.append([n, experimento, es_secuencial, cantThreadsLectura, cantThreadsMaximo, tiempo]);
n += 1;

for f in range(1,30):
    os.remove(F"../data/corpus{f}")

df_resultado = pd.DataFrame(filas, columns=columnas);
df_resultado.to_csv("../resultados/resultado_cargarArchivo_secuencial.csv", index=False, header=True);

'Experimento: 1'

## Experimento cargar archivos concurrente

In [19]:
columnas = ["n", "experimento", "es_secuencial", "cantThreadsLectura", "cantThreadsMaximo", "tiempo"];
filas = [];

n = 1;
experimento = "cargarArchivos";
es_secuencial = "No";
cantThreadsMaximo = 1;
tiempos = []
filePaths = ["../data/corpus"]
for f in range(1,30):
    sh.copyfile("../data/corpus",F"../data/corpus{f}");
    filePaths.append(F"../data/corpus{f}")

for cantThreadsLectura in range(1,31):
    for t in range(0, 5):
        analisis = correr_experimento(experimento, es_secuencial, cantThreadsLectura, cantThreadsMaximo, filePaths);
        tiempos.append(analisis);
    clear_output(wait=True)
    display('Experimento: ' + str(n))
    tiempo = np.median(tiempos);
    filas.append([n, experimento, es_secuencial, cantThreadsLectura, cantThreadsMaximo, tiempo]);
    n += 1;

for f in range(1,30):
    os.remove(F"../data/corpus{f}")

df_resultado = pd.DataFrame(filas, columns=columnas);
df_resultado.to_csv("../resultados/resultado_cargarArchivo_concurrente.csv", index=False, header=True);

'Experimento: 30'